# Sample 2

This notebook demonstrate how to use Azure Eval SDK to locally run evaluations using the custom evaluators.

In [ ]:
import os
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.evaluation import evaluate, RelevanceEvaluator, EvaluatorConfig, AzureAIProject
from custom_evaluator.friendliness_evaluator import FriendlinessEvaluator
from custom_evaluator.length_evaluator import LengthEvaluator


In [ ]:
load_dotenv()

model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

### Configure evaluators

In [ ]:
evaluators = {
    "relevance": RelevanceEvaluator(model_config=model_config),
    "friendliness": FriendlinessEvaluator(model_config=model_config),
    "length": LengthEvaluator()
}

evaluator_config = {
    "relevance": EvaluatorConfig(
        column_mapping={
            "query": "${data.question}",
            "response": "${data.answer}"
        }
    ),
    "friendliness": EvaluatorConfig(
        column_mapping={
            "response": "${data.answer}"
        }
    ),
    "length": EvaluatorConfig(
        column_mapping= {
            "response": "${data.answer}"
        } 
)
}

### Run evaluation

In [ ]:
results = evaluate(
    data="data/qna.jsonl",
    evaluators=evaluators,
    evaluator_config=evaluator_config,
    evaluation_name="Local Evaluation Run",
    fail_on_evaluator_errors=True,
    output_path="output/custom_batch_eval.json"
)

print(f"Results: {results}")

### Run evaluation w/ upload results to cloud

In [ ]:
# optional - to support DefaultAzureCredential()
!az login

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["AI_FOUNDRY_CONNECTION_STRING"]
)

azure_project = AzureAIProject(
    subscription_id=project_client.scope["subscription_id"],
    resource_group_name=project_client.scope["resource_group_name"],
    project_name=project_client.scope["project_name"]
)

results = evaluate(
    data="data/qna.jsonl",
    evaluators=evaluators,
    evaluator_config=evaluator_config,
    azure_ai_project=azure_project,  
    evaluation_name="Local Evaluation Run",
    fail_on_evaluator_errors=True,
)

print(f"Results: {results}")